# Explicación de Redes Neuronales Recurrentes (RNN)

En este cuaderno, explicaremos cómo funcionan las Redes Neuronales Recurrentes (RNN), sus aplicaciones, limitaciones y cómo se comparan con otras arquitecturas como LSTM y Transformers.

## Objetivos
- Comprender el funcionamiento básico de una RNN.
- Explorar cómo las RNN procesan secuencias de datos, especialmente en tareas de texto.
- Discutir las limitaciones de las RNN y sus desafíos.
- Comparar las RNN con LSTM y Transformers.

## 1. Introducción a las RNN
Las RNN son un tipo de red neuronal diseñada para trabajar con datos secuenciales, como texto, series temporales o secuencias de video. Su estructura les permite recordar información de estados anteriores, lo que las hace útiles para tareas en las que el contexto importa.

### Funcionamiento básico de una RNN
En una RNN, la salida de una celda recurrente en un paso temporal es pasada como entrada a la celda en el siguiente paso. Esto permite que la red acumule información a medida que avanza por la secuencia. 

![Arquitectura básica de una RNN](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/2880px-Recurrent_neural_network_unfold.svg.png)

En la imagen, podemos ver cómo la misma celda se repite para cada paso en la secuencia, compartiendo los mismos pesos y acumulando el contexto a lo largo del tiempo.

## 2. Procesamiento de Secuencias en una RNN
Veamos un ejemplo de cómo una RNN procesa una secuencia de texto para predecir la siguiente palabra en la secuencia. Usaremos un conjunto simple de oraciones.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
import numpy as np

# Datos de ejemplo
sentences = ["el gato juega", "el perro corre", "la luna brilla"]
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
max_sequence_len = max([len(x) for x in sequences])
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_len)

# Dividir en características (X) y etiquetas (y)
X, y = sequences[:, :-1], sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index)+1)

# Crear el modelo RNN
model_rnn = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=max_sequence_len-1),
    SimpleRNN(32),
    Dense(len(tokenizer.word_index)+1, activation='softmax')
])

# Compilar el modelo
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_rnn.summary()

### Entrenamiento de la RNN
Entrenamos el modelo con las secuencias anteriores.

In [ ]:
# Entrenar el modelo
history_rnn = model_rnn.fit(X, y, epochs=200, verbose=1)


### Visualización del rendimiento
Mostramos las curvas de precisión y pérdida del modelo durante el entrenamiento.

In [ ]:
import matplotlib.pyplot as plt
# Graficar la precisión y la pérdida
plt.plot(history_rnn.history['accuracy'], label='Precisión')
plt.plot(history_rnn.history['loss'], label='Pérdida')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.title('Rendimiento del modelo RNN')
plt.legend()
plt.show()

## 3. Limitaciones de las RNN
Las RNN enfrentan varios desafíos, entre ellos:
- **Desvanecimiento del gradiente**: A medida que la secuencia se hace más larga, las RNN tienen dificultades para recordar información de los primeros pasos, lo que limita su capacidad para manejar dependencias a largo plazo.
- **Paralelización**: Debido a la naturaleza secuencial de las RNN, es difícil paralelizar su entrenamiento, lo que las hace más lentas en comparación con otros modelos más recientes.


### Ejemplo de desvanecimiento de gradiente
Probemos con secuencias más largas para ver cómo afecta la precisión de las predicciones.

In [ ]:
# Secuencias más largas
long_sentences = ["el gato juega en el jardín mientras el sol brilla", "la luna llena ilumina el cielo nocturno"]
long_sequences = tokenizer.texts_to_sequences(long_sentences)
long_sequences = tf.keras.preprocessing.sequence.pad_sequences(long_sequences, maxlen=max_sequence_len)

# Predicciones del modelo
predictions = model_rnn.predict(long_sequences)
predicted_words = [np.argmax(pred) for pred in predictions]
predicted_words_text = [list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(idx)] for idx in predicted_words]

print(f"Secuencias de entrada: {long_sentences}")
print(f"Palabras predichas: {predicted_words_text}")

## 4. Comparación con LSTM y Transformers
Las LSTM y GRU son variantes de las RNN que abordan algunos de estos problemas, como el desvanecimiento del gradiente. Los Transformers, por otro lado, eliminan completamente la dependencia secuencial, lo que permite un procesamiento paralelo y mayor eficiencia.

### LSTM
Las LSTM utilizan celdas de memoria y puertas de entrada, olvido y salida para decidir qué información mantener o desechar. Esto les permite manejar dependencias a largo plazo de manera más efectiva.

### Transformers
Los Transformers utilizan mecanismos de atención que permiten a cada posición en una secuencia de entrada relacionarse directamente con cualquier otra posición, eliminando la necesidad de procesar los datos en orden secuencial.

En un cuaderno futuro, implementaremos estas variantes para comparar su rendimiento con las RNN.